In [ ]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
import datetime
from pathlib import Path

TABLE_ID = '80072ned'
TABLE_PATH = Path(f'./data/{TABLE_ID}')
GRAPH_PATH = Path(f'./plots/{TABLE_ID}_{datetime.datetime.now().strftime("%Y%m%dT%H%M%S")}')

GRAPH_PATH.mkdir(exist_ok=True, parents=True)

def get_figure_number():
    figure_number = 0
    while True:
        yield figure_number
        figure_number += 1
figure_number_generator = get_figure_number() 

def get_graph_name(title: str) -> str:
    return f'{next(figure_number_generator):0=3d}_{title}'
    
def save_graph(graph: alt.Chart, title: str):
    graph_name = get_graph_name(title)
    graph.save(GRAPH_PATH / f'{graph_name}.svg')
    graph.save(GRAPH_PATH / f'{graph_name}.png')
        
def save_fig(figure, title: str):
    graph_name = get_graph_name(title)
    figure.savefig(GRAPH_PATH / f'{graph_name}.svg')
    figure.savefig(GRAPH_PATH / f'{graph_name}.png')
    
# required for export to pdf with images?
alt.renderers.enable('png')


# UWV Exploratory Analysis

In [ ]:
slp: pd.DataFrame = pd.read_parquet(TABLE_PATH / f'{TABLE_ID}.parquet')
slp.info()

In [ ]:
slp

In [ ]:
slp.category_group_title.value_counts()

## Get train and test

We will use 2022 and up as the final test data.
All prior tot 2022 will be training data. To test the trained model, we wil use 2021. So we get three splits:

- All data from 2013 onward and prior to 2021 is the real train data. This is the data to perform exploratory data analysis on. 
- All data from 2021 wil be the test set to test our trained models on. 
- When we are really done, 2022 and onwards will be the final test set. 

Additionally, we will only use the quarterly numbers (period_type = 'KW')

In [ ]:
slp_test = slp[(slp.period_year >= 2022) & (slp.period_type == 'KW')]
slp_train = slp[(slp.period_year > 2012) & (slp.period_year < 2021) & (slp.period_type == 'KW')]
slp_train_test = slp[(slp.period_year == 2021) & (slp.period_type == 'KW')]

In [ ]:
slp_train.info()

In [ ]:
c = alt.Chart(slp_train).mark_line().encode(
    x='period', 
    y='sick_leave_percentage',
    color='sbi_title'
)

save_graph(c, 'slp_train_all_sbi')
c

In [ ]:
c = alt.Chart(slp_train[slp_train.category_group_title.isin(['Bedrijfsgrootte', 'Totaal'])]).mark_line().encode(
    x='period', 
    y='sick_leave_percentage',
    color='sbi_title'
)

save_graph(c, 'slp_train_sbi_size')
c

In [ ]:
c = alt.Chart(slp_train[slp_train.category_group_title.isin(['Bedrijfssector', 'Totaal'])]).mark_line().encode(
    x='period', 
    y='sick_leave_percentage',
    color='sbi_title'
)

save_graph(c, 'slp_train_sbi_sector')
c

In [ ]:
c = alt.Chart(slp_train[slp_train.category_group_title.isin([ 'Bedrijfsklasse', 'Totaal'])]).mark_line().encode(
    x='period', 
    y='sick_leave_percentage',
    color='sbi_title'
)

save_graph(c, 'slp_train_sbi_klasse')
c

In [ ]:
c = alt.Chart(slp_train[slp_train.category_group_title.isin(['Bedrijfstak', 'Totaal'])]).mark_line().encode(
    x='period', 
    y='sick_leave_percentage',
    color='sbi_title'
)

save_graph(c, 'slp_train_sbi_tak')
c

In [ ]:
c = alt.Chart(slp_train).mark_boxplot().encode(
    x='period',
    y='sick_leave_percentage',
)

save_graph(c, 'slp_train_boxplot')
c

In [ ]:
slp_train_total = slp_train[slp_train.sbi == 'T001081']

c = alt.Chart(slp_train_total).mark_line().encode(
    x='period_quarter_number',
    y='sick_leave_percentage',
    color='period_year'
).properties(title='Seasonality of T001081 sick leave %')

save_graph(c, 'slp_train_total_seasonality')
c

In [ ]:
c = alt.Chart(slp_train).mark_point().encode(
     x=alt.X(alt.repeat("column"), type='ordinal'),
    y='sick_leave_percentage',
    color='period_year'   
).repeat(
    column=['period_quarter_number']
).properties(
    title='Seasonality sick leave % (all categories)'
)

save_graph(c, 'slp_train_seasonality_all_sbi')
c

In [ ]:
c = alt.Chart(slp_train_total).mark_point().encode(
    x=alt.X(alt.repeat("column"), type='ordinal'),
    y='sick_leave_percentage',
    color='period_year'
).repeat(
    column=['period_quarter_number']
).properties(
    title='Seasonality of T001081 sick leave %'
)

save_graph(c, 'slp_train_total_seasonality_point')
c

In [ ]:
c = alt.Chart(slp_train[slp_train.period_quarter_number == 1]).mark_point().encode(
    x='period',
    y='sick_leave_percentage',
    color='period_year'
) | alt.Chart(slp_train[slp_train.period_quarter_number  == 2]).mark_point().encode(
    x='period',
    y='sick_leave_percentage',
    color='period_year'
) | alt.Chart(slp_train[slp_train.period_quarter_number  == 3]).mark_point().encode(
    x='period',
    y='sick_leave_percentage',
    color='period_year'
) | alt.Chart(slp_train[slp_train.period_quarter_number  == 4]).mark_point().encode(
    x='period',
    y='sick_leave_percentage',
    color='period_year'
)

save_graph(c, 'slp_train_seasonality_all_sbi_point_per_quarter')
c

In [ ]:

c = alt.Chart(slp_train_total[slp_train_total.period_quarter_number ==  1]).mark_point().encode(
    x='period',
    y='sick_leave_percentage',
    color='period_year'
) | alt.Chart(slp_train_total[slp_train_total.period_quarter_number ==  2]).mark_point().encode(
    x='period',
    y='sick_leave_percentage',
    color='period_year'
) | alt.Chart(slp_train_total[slp_train_total.period_quarter_number ==  3]).mark_point().encode(
    x='period',
    y='sick_leave_percentage',
    color='period_year'
) | alt.Chart(slp_train_total[slp_train_total.period_quarter_number ==  4]).mark_point().encode(
    x='period',
    y='sick_leave_percentage',
    color='period_year'
)

save_graph(c, 'slp_train_total_seasonality_point_per_quarter')
c

In [ ]:
from pandas.plotting import lag_plot

fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20,10))

for i, lag in enumerate(range(1,9)):
    lag_plot(slp_train_total.sick_leave_percentage, lag=lag, ax=axes[i // 4, i % 4])
   
save_fig(plt, 'slp_train_total_lag_plot')
plt.show()

In [ ]:
import math

start_lag = 0
lag_length = 21

lagged_auto_correlation = pd.DataFrame()
lagged_auto_correlation['lag'] = range(start_lag, lag_length)

white_noise_border = 1.96 / math.sqrt(len(slp_train_total.sick_leave_percentage))

wn_border = pd.DataFrame()
wn_border['lag'] = range(start_lag - 1, lag_length + 1)
wn_border['pos_white_noise_border'] = [white_noise_border for _ in range(start_lag - 1, lag_length + 1)]
wn_border['neg_white_noise_border'] = [-white_noise_border for _ in range(start_lag - 1, lag_length + 1)]

lagged_auto_correlation['auto_correlation'] = [slp_train_total.sick_leave_percentage.autocorr(lag=lag) for lag in lagged_auto_correlation['lag']]

c = alt.Chart(lagged_auto_correlation).mark_bar().encode(
    x='lag',
    y='auto_correlation',
) + alt.Chart(wn_border).mark_line(strokeDash=[1,1]).encode(
    x='lag',
    y='pos_white_noise_border',
) + alt.Chart(wn_border).mark_line(strokeDash=[1,1]).encode(
    x='lag',
    y='neg_white_noise_border'
)

save_graph(c, 'slp_train_total_auto_correlation')
c

In [ ]:
moving_average = pd.DataFrame()

moving_average['quarter'] = slp_train_total.period
moving_average['sick'] = slp_train_total.sick_leave_percentage

for window in range(3, 16, 2):
    moving_average[f'{window}-MA'] = slp_train_total.sick_leave_percentage.rolling(window, center=True).mean()

moving_average


In [ ]:
charts = [alt.Chart(moving_average).mark_line().encode(x='quarter', y='sick')]

for window in range(3, 16, 2):
    charts.append(alt.Chart(moving_average).mark_line().encode(x='quarter', y=f'{window}-MA'))
    
c = alt.vconcat(*charts)
save_graph(c, 'slp_train_total_moving_average')
c

In [ ]:
from statsmodels.tsa.seasonal import STL

slp_series = slp_train_total.sick_leave_percentage
slp_series.index = slp_train_total.period
slp_series

In [ ]:
plt.rc("font", size=6)
stl = STL(slp_series, period=4)
res = stl.fit()

fig = res.plot()
fig.autofmt_xdate()
save_fig(fig, 'slp_train_total_STL_plot')
fig

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

plot_pacf(slp_series, lags=15, alpha=0.1)
save_fig(plt, 'slp_train_total_pacf')
plt.show()